In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit
import networkx as nx


In [2]:
import sys
sys.path.append('../../code')

from metrics import get_hi_metrics
import min_vertex_k_cut

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


# Prepare dataset

In [3]:
train = pd.read_csv('../../data/hi/drd2/train_1.csv', index_col=0)
test = pd.read_csv('../../data/hi/drd2/test_1.csv', index_col=0)

train

,smiles,value
383,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,True
386,CC(C)Oc1ccccc1N1CCN(Cc2cccc(CN3CCCCC3=O)c2)CC1,True
389,CC(C)Oc1ccccc1N1CCN(Cc2ccccc2CN2CCCCC2=O)CC1,True
2695,COc1ccccc1N1CCN(CC2COCC(c3ccccc3)(c3ccccc3)O2)CC1,True
2995,COc1ccccc1N1CCN(C[C@H]2OCCOC2(c2ccccc2)c2ccccc...,False
...,...,...
5444,O=C1c2ccccc2C(=O)N1CCCCN1CCCN(C(c2ccccc2)c2ccc...,True
4391,O=C(CCC(=O)c1ccccc1)NCCc1c[nH]c2ccccc12,False
4397,O=C(CCCC(=O)c1ccccc1)NCCc1c[nH]c2ccccc12,False
5999,OC12C3C4CC5C6C4C1C6C(C53)N2CC1CCCCC1,False


# Split train into train and val

In [4]:
smiles = train['smiles'].to_list()
threshold = 0.4

neighborhood_graph = min_vertex_k_cut.get_neighborhood_graph(smiles, threshold)
main_component, small_components = min_vertex_k_cut.get_main_component(neighborhood_graph)

old_nodes_to_new = dict(zip(main_component.nodes(), range(main_component.number_of_nodes())))
new_nodes_to_old = {v: k for k, v in old_nodes_to_new.items()}
main_component = nx.relabel_nodes(main_component, old_nodes_to_new)

In [5]:
coarsed_main_component, node_to_cluster = min_vertex_k_cut.coarse_graph(main_component, 0.4)

In [6]:
model = min_vertex_k_cut.train_test_split_connected_graph(coarsed_main_component, train_min_fraq=0.5, test_min_fraq=0.1, max_mip_gap=0.3)

Total molecules: 1193
Min train size 596
Min test size 119
Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 391 (-178) rows, 178 (0) columns and 956 (-356) elements
Clp1000I sum of infeasibilities 1.71113e-07 - average 4.37629e-10, 0 fixed columns
Coin0506I Presolve 391 (0) rows, 178 (0) columns and 956 (0) elements
Clp0029I End of values pass after 178 iterations
Clp0014I Perturbing problem by 0.001% of 0.67057672 - largest nonzero change 2.989373e-05 ( 0.00242536%) - largest zero change 0
Clp0000I Optimal - objective value 1193
Clp0000I Optimal - objective value 1193
Clp0000I Optimal - objective value 1193
Coin0511I After Postsolve, objective 1193, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 1193 - 0 iterations time 0.022, Presolve 0.00, Idiot 0.02

Starting MIP optimization
Cgl0003I 5 fixed, 0 tightened bounds, 37 strengthened rows

In [7]:
split = min_vertex_k_cut.process_bisect_results(model, coarsed_main_component, main_component, node_to_cluster)

Molecules in train: 1039
Molecules in test: 133
Molecules lost: 21


In [8]:
first_idx = []
second_idx = []

for S_idx, partition in enumerate(split):
    G_idx = new_nodes_to_old[S_idx]
    if partition == 0:
        first_idx.append(G_idx)
    if partition == 1:
        second_idx.append(G_idx)

In [9]:
for component in small_components:
    i = np.argmin([len(first_idx), len(second_idx)])
    if i == 0:
        first_idx.extend(component)
    if i == 1:
        second_idx.extend(component)

In [10]:
print(len(first_idx))
print(len(second_idx))

1180
1184


In [11]:
part_first = train.iloc[first_idx]
part_second = train.iloc[second_idx]

## Hi split

In [12]:
from sklearn.metrics import average_precision_score

def run_gb_gridsearch(train_fps, val_fps, train_y, val_y):
    split_index = [-1] * len(train_fps) + [0] * len(val_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + val_fps
    y = train_y + val_y

    params = {
    'n_estimators': [10, 50, 100, 150, 200, 250, 500],
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7, 1.0],
    'subsample': [0.4, 0.7, 0.9, 1.0],
    'min_samples_split': [2, 3, 5, 7],
    'min_samples_leaf': [1, 3, 5],
    'max_depth': [2, 3, 4],
    'max_features': [None, 'sqrt']
    }
    gb = GradientBoostingClassifier()

    grid_search = RandomizedSearchCV(gb, params, cv=pds, n_iter=30, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    gb = GradientBoostingClassifier(**best_params)
    gb.fit(train_fps, train_y)

    val_preds = gb.predict_proba(val_fps)[:, 1]
    val_metrics = average_precision_score(val_y, val_preds)
    return val_metrics


In [13]:
train_mols = [Chem.MolFromSmiles(x) for x in part_first['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

val_mols = [Chem.MolFromSmiles(x) for x in part_second['smiles']]
val_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in val_mols]

test_metrics = run_gb_gridsearch(train_morgan_fps, val_morgan_fps, part_first['value'].to_list(), part_second['value'].to_list())
print(test_metrics)

Fitting 1 folds for each of 30 candidates, totalling 30 fits
[CV 1/1] END learning_rate=0.7, max_depth=2, max_features=sqrt, min_samples_leaf=3, min_samples_split=2, n_estimators=50, subsample=0.4;, score=0.669 total time=   1.3s
[CV 1/1] END learning_rate=0.1, max_depth=2, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100, subsample=0.4;, score=0.625 total time=   1.4s
[CV 1/1] END learning_rate=0.7, max_depth=2, max_features=None, min_samples_leaf=5, min_samples_split=2, n_estimators=200, subsample=1.0;, score=0.648 total time=   4.8s
[CV 1/1] END learning_rate=0.5, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=250, subsample=1.0;, score=0.669 total time=   1.5s
[CV 1/1] END learning_rate=0.5, max_depth=2, max_features=None, min_samples_leaf=3, min_samples_split=5, n_estimators=10, subsample=0.7;, score=0.666 total time=   1.2s
[CV 1/1] END learning_rate=0.7, max_depth=3, max_features=sqrt, min_samples_leaf=5, min_sam

### Final Evaluation

In [21]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

gb = GradientBoostingClassifier(
    subsample=0.7,
    n_estimators=150,
    min_samples_split=3,
    min_samples_leaf=3,
    max_features=None,
    max_depth=2,
    learning_rate=0.5
)

gb.fit(train_morgan_fps, train['value'].to_list())
test_preds = gb.predict_proba(test_morgan_fps)[:, 1]
test_metrics = average_precision_score(test['value'], test_preds)
print(test_metrics)


0.6770894792792055


# Scaffold split

In [15]:
from rdkit.ML.Cluster import Butina
from numpy.random import default_rng


def butina_split(smiles: list[str], cutoff: float, seed: int, frac_train=0.8):
    """
    Select distinct molecules to train/test. Returns indices of the molecules in the smiles list.
    Adapted from DeepChem (https://deepchem.io/), but random seed is added.
    """

    mols = [Chem.MolFromSmiles(smile) for smile in smiles]
    fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in mols]

    dists = []
    nfps = len(fps)
    for i in range(1, nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fps[i], fps[:i])
        dists.extend([1 - x for x in sims])
    scaffold_sets = Butina.ClusterData(dists, nfps, cutoff, isDistData=True)
    scaffold_sets = sorted(scaffold_sets, key=lambda x: -len(x))

    rng = default_rng(seed)
    rng.shuffle(scaffold_sets)

    train_cutoff = frac_train * len(smiles)
    train_inds = []
    test_inds = []

    for scaffold_set in scaffold_sets:
        if len(train_inds) + len(scaffold_set) > train_cutoff:
            test_inds += scaffold_set
        else:
            train_inds += scaffold_set
    return train_inds, test_inds

In [16]:
train_idx, val_idx = butina_split(train['smiles'].to_list(), cutoff=0.5, seed=123, frac_train=0.5)

In [17]:
part_first = train.iloc[train_idx]
part_second = train.iloc[val_idx]

In [18]:
from sklearn.metrics import average_precision_score

def run_gb_gridsearch(train_fps, val_fps, train_y, val_y):
    split_index = [-1] * len(train_fps) + [0] * len(val_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + val_fps
    y = train_y + val_y

    params = {
    'n_estimators': [10, 50, 100, 150, 200, 250, 500],
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7, 1.0],
    'subsample': [0.4, 0.7, 0.9, 1.0],
    'min_samples_split': [2, 3, 5, 7],
    'min_samples_leaf': [1, 3, 5],
    'max_depth': [2, 3, 4],
    'max_features': [None, 'sqrt']
    }
    gb = GradientBoostingClassifier()

    grid_search = RandomizedSearchCV(gb, params, cv=pds, n_iter=30, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    gb = GradientBoostingClassifier(**best_params)
    gb.fit(train_fps, train_y)

    val_preds = gb.predict_proba(val_fps)[:, 1]
    val_metrics = average_precision_score(val_y, val_preds)
    return val_metrics


In [19]:
train_mols = [Chem.MolFromSmiles(x) for x in part_first['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

val_mols = [Chem.MolFromSmiles(x) for x in part_second['smiles']]
val_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in val_mols]

test_metrics = run_gb_gridsearch(train_morgan_fps, val_morgan_fps, part_first['value'].to_list(), part_second['value'].to_list())
print(test_metrics)

Fitting 1 folds for each of 30 candidates, totalling 30 fits
[CV 1/1] END learning_rate=0.01, max_depth=4, max_features=sqrt, min_samples_leaf=5, min_samples_split=3, n_estimators=200, subsample=1.0;, score=0.877 total time=   2.0s
[CV 1/1] END learning_rate=0.5, max_depth=2, max_features=None, min_samples_leaf=5, min_samples_split=3, n_estimators=250, subsample=0.9;, score=0.859 total time=   5.4s
[CV 1/1] END learning_rate=0.3, max_depth=3, max_features=None, min_samples_leaf=1, min_samples_split=3, n_estimators=10, subsample=0.9;, score=0.836 total time=   1.4s
[CV 1/1] END learning_rate=0.3, max_depth=2, max_features=sqrt, min_samples_leaf=3, min_samples_split=5, n_estimators=500, subsample=0.7;, score=0.831 total time=   1.8s
[CV 1/1] END learning_rate=1.0, max_depth=4, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100, subsample=0.9;, score=0.809 total time=   4.5s
[CV 1/1] END learning_rate=1.0, max_depth=4, max_features=None, min_samples_leaf=5, min_s

### Final Evaluation

In [22]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

gb = GradientBoostingClassifier(
    subsample=1.0,
    n_estimators=200,
    min_samples_split=3,
    min_samples_leaf=5,
    max_features='sqrt',
    max_depth=4,
    learning_rate=0.01
)

gb.fit(train_morgan_fps, train['value'].to_list())
test_preds = gb.predict_proba(test_morgan_fps)[:, 1]
test_metrics = average_precision_score(test['value'], test_preds)
print(test_metrics)


0.6629331875367845
